### Benchmarking using the 2023 EBA Stresstest Database

Welcome to this Jupyter notebook, where we delve into the European Banking Authority (EBA) stresstest database for the year 2023.

**Here's a brief overview of what we'll be doing:**

1. **Data Import:** We start by importing the EBA stresstest database from 2023. This provides us with a comprehensive set of data related to the financial health and resilience of major banks during hypothetical adverse market conditions.
  
2. **Risk Metrics Creation:** Using the raw data, we derive additional risk metrics that give us a clearer picture of the banks' exposure and their ability to weather financial downturns.

3. **Decile Analysis:** For each risk metric, we calculate deciles. This statistical measure divides the data into ten equal parts, enabling us to categorize and understand the distribution of risk across banks.

4. **Bank vs. Industry Comparison:** Lastly, we pick a single bank and contrast its risk profile with the broader industry data. This helps in gauging where the bank stands relative to its peers.

Dive in to explore the insights and trends from the 2023 stresstest!

The dataset has been obtained from https://www.eba.europa.eu/risk-analysis-and-data/eu-wide-stress-testing#collapseTwoB

In [1]:
import os
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import numpy as np

import psycopg2
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import ForeignKey, Integer, Float, String, Column
from sqlalchemy.orm import sessionmaker
from sqlalchemy.schema import CreateSchema
from sqlalchemy import create_engine, MetaData
from sqlalchemy.exc import ProgrammingError
from sqlalchemy import inspect

### A. Parametrisation

In [2]:
# Folders
path = os.getcwd()
input_data = os.path.join(path, 'input_data')
output_data = os.path.join(path, 'output_data')

### B. Import data

In [3]:
def whitespace_remover(df):
    # replace double spaces
    for i in df.columns:
        if df[i].dtype == 'object':             
            df[i] = df[i].str.strip().str.replace('  ', ' ')

    return df
            
def retrieve_metadata(path, sheet_name=None):
    # Retrieve tables with metadata 
    df = pd.read_excel(path, sheet_name=sheet_name)
    
    # Convert all labels to lowercase, joined by underscore
    df.columns = df.columns.str.replace(' ', '_').str.replace('  ', ' ').str.lower()
    df = whitespace_remover(df)
    df.rename(columns={'template_no#': 'template'}, inplace=True)

    if sheet_name == "SDD":
        metadata_dict = dict(zip(df['item'], df['label']))
    else:
        metadata_dict = dict(zip(df.iloc[:, 0], df.iloc[:, 1]))
    
    return metadata_dict


def import_fact_tables(input_data, file):
    # Import and transform fact tables
    
    df = pd.read_csv(os.path.join(input_data, file), 
                     # encoding='unicode_escape', 
                     encoding='utf-8',
                     low_memory=False)
    
    # Convert all labels to lowercase, joined by underscore
    df.columns = df.columns.str.replace(' ', '_').str.strip().str.lower()
    
    # EBA assigns '.' to missing values. Replace if it's the only character
    df = df.where(df != '.', other=np.nan)
    
    # Convert string to numeric (todo: consider handling coerced errors)
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
    df['amount'] = df['amount'].replace(0, np.nan)

    # Other
    df = whitespace_remover(df)
    df['period'] = pd.to_datetime(df['period'], format='%Y%m') + MonthEnd(0)
    df['period_str'] = df['period'].dt.strftime('%Y_%m')
    
    return df

In [4]:
# Prepare paths
metadata_path = os.path.join(input_data, 'Metadata_TR.xlsx')
itemdata_path = os.path.join(input_data, 'Data_Dictionary.xlsx')

# Import files with dimension tables
country = retrieve_metadata(metadata_path, sheet_name='Country')
scenario = retrieve_metadata(metadata_path, sheet_name='Scenario')
portfolio = retrieve_metadata(metadata_path, sheet_name='Portfolio')
exposure = retrieve_metadata(metadata_path, sheet_name='Exposure')
status = retrieve_metadata(metadata_path, sheet_name='Status')
ifrs9_stages = retrieve_metadata(metadata_path, sheet_name='IFRS9_Stages')
cr_guarantees = retrieve_metadata(metadata_path, sheet_name='CR_guarantees')
item = retrieve_metadata(itemdata_path, sheet_name='SDD')

# Perform transformations
tra_oth = import_fact_tables(input_data, 'TRA_OTH.csv')
tra_cre_sta = import_fact_tables(input_data, 'TRA_CRE_STA.csv')
tra_cre_irb = import_fact_tables(input_data, 'TRA_CRE_IRB.csv')
tra_cre_cov = import_fact_tables(input_data, 'TRA_CRE_COV.csv')

# Replace columns in the facts table
tra_oth['item'] = tra_oth['item'].map(item)
tra_oth['scenario'] = tra_oth['scenario'].map(scenario)

tra_cre_irb['scenario'] = tra_cre_irb['scenario'].map(scenario)
tra_cre_irb['item'] = tra_cre_irb['item'].map(item)
tra_cre_irb['portfolio'] = tra_cre_irb['portfolio'].map(portfolio)
tra_cre_irb['country'] = tra_cre_irb['country'].map(country)
tra_cre_irb['exposure'] = tra_cre_irb['exposure'].map(exposure)

tra_cre_irb['ifrs9_stages'] = tra_cre_irb['ifrs9_stages'].map(ifrs9_stages)
tra_cre_irb['status'] = tra_cre_irb['status'].map(status)
tra_cre_irb['cr_guarantees'] = tra_cre_irb['cr_guarantees'].map(cr_guarantees)

In [5]:
# Banks participating in this exercise
tra_oth['bank_name'].unique()

array(['Erste Group Bank AG', 'Raiffeisen Bank International AG',
       'Belfius Banque SA', 'KBC Group NV',
       'Deutsche Apotheker- und Ärztebank eG',
       'Citigroup Global Markets Europe AG',
       'COMMERZBANK Aktiengesellschaft', 'Deutsche Bank AG',
       'DZ Bank AG Deutsche Zentral-Genossenschaftsbank',
       'Goldman Sachs Bank Europe SE', 'HASPA Finanzholding',
       'J.P. Morgan SE', 'Landesbank Baden-Württemberg',
       'Bayerische Landesbank',
       'Landesbank Hessen-Thüringen Girozentrale',
       'Morgan Stanley Europe Holding SE',
       'Norddeutsche Landesbank -Girozentrale-', 'Volkswagen Bank GmbH',
       'Danske Bank A/S', 'Jyske Bank A/S', 'Nykredit Realkredit A/S',
       'Sydbank A/S', 'Banco Bilbao Vizcaya Argentaria, S.A.',
       'Bankinter, S.A.', 'CaixaBank, S.A.', 'Kutxabank, S.A.',
       'ABANCA Corporación Bancaria S.A.', 'Banco de Sabadell, S.A.',
       'Banco Santander, S.A.', 'Unicaja Banco, S.A.', 'Nordea Bank Abp',
       'OP Osuuskun

### C. Create ratios based on the general template

In [6]:
# Create the analysis table
idx_1 = tra_oth['item'].isin(['Net interest income', 
                              'Common Equity Tier 1 ratio %', 
                              'Net fee and commission income', 
                              'TOTAL RISK EXPOSURE AMOUNT',
                              'Leverage ratio %'])

# Pivot table
other_df = tra_oth[idx_1].pivot_table(index=['bank_name', 'item'], 
                                      columns=['period_str', 'scenario'], 
                                      values='amount', 
                                      aggfunc='sum').reset_index()
other_df.columns = ['_'.join(map(str, col)) if type(col) is tuple 
                                            else col for col in other_df.columns]
other_df.columns = [s.rstrip('_').replace(' ', '_') for s in other_df.columns]

other_df.sample(10)

,bank_name,item,2022_12_Actual_figures,2023_12_Adverse_scenario,2023_12_Baseline_scenario,2024_12_Adverse_scenario,2024_12_Baseline_scenario,2025_12_Adverse_scenario,2025_12_Baseline_scenario
173,Goldman Sachs Bank Europe SE,Net interest income,6.796828,368.891339,1097.684680,367.028313,984.936489,365.323853,750.027636
185,HASPA Finanzholding,Common Equity Tier 1 ratio %,0.152676,0.137728,0.148721,0.126319,0.150659,0.123318,0.153944
290,Powszechna Kasa Oszczednosci Bank Polski S.A.,Common Equity Tier 1 ratio %,0.176715,0.136227,0.188885,0.132275,0.205839,0.132587,0.222697
322,Swedbank — group,Net fee and commission income,1230.669586,861.470438,1208.517533,861.470513,1197.441507,861.470590,1209.748203
229,La Banque Postale,TOTAL RISK EXPOSURE AMOUNT,92798.302161,92770.540433,93970.524511,95525.461931,95978.844353,98310.088785,97962.165791
311,Société Générale S.A.,Leverage ratio %,0.043668,0.034965,0.044055,0.034348,0.044093,0.033189,0.043119
342,Volkswagen Bank GmbH,Net fee and commission income,272.329000,189.945010,252.666846,189.870715,246.457745,189.791493,238.805300
221,"Kutxabank, S.A.",Leverage ratio %,0.077112,0.066922,0.084385,0.067121,0.091824,0.067883,0.095887
95,COMMERZBANK Aktiengesellschaft,Common Equity Tier 1 ratio %,0.141374,0.102401,0.144633,0.098441,0.149407,0.094940,0.152453
208,J.P. Morgan SE,Net interest income,1069.984826,1305.775653,3283.992163,1302.698735,4502.050136,1302.132962,4243.606623


In [7]:
# Initialize an empty list to store the new rows
new_rows = []

# Calculate ratios for Other template
for _, row in other_df.iterrows():
    # Exclude observations where calculating ratio is not possible
    if (row['2022_12_Actual_figures'] in {0, None}) | (row['2023_12_Adverse_scenario'] in {0, None}):
        continue
    
    elif row['item'] == 'Common Equity Tier 1 ratio %':
        new_rows.append([row['bank_name'], 'CET1_ratio_adverse_Y1', row['2023_12_Adverse_scenario']])
        
    elif row['item'] == 'Net fee and commission income':
        ratio =  row['2023_12_Adverse_scenario'] / row['2023_12_Baseline_scenario']
        new_rows.append([row['bank_name'], 'Net_fee_and_commission_income_adverse_vs_baseline_Y1', ratio])

    elif row['item'] == 'Net interest income':
        ratio =  row['2023_12_Adverse_scenario'] / row['2023_12_Baseline_scenario']
        new_rows.append([row['bank_name'], 'Net_interest_income_adverse_vs_baseline_Y1', ratio])

    elif row['item'] == 'TOTAL RISK EXPOSURE AMOUNT':
        ratio =  row['2023_12_Baseline_scenario'] / row['2023_12_Adverse_scenario']
        new_rows.append([row['bank_name'], 'TREA_baseline_vs_adverse_Y1', ratio])

    elif row['item'] == 'Leverage ratio %':
        new_rows.append([row['bank_name'], 'Leverage_ratio_adverse_Y1', row['2023_12_Adverse_scenario']])

    else:
        new_rows.append(['Problem', 'Problem', 0])

# Convert the list to a new dataframe
general_df = pd.DataFrame(new_rows, columns=['bank_name', 'Metric', 'value'])

### D. Create ratios based on the IRB  template

In [8]:
# Get IRB losses
col = 'Corporates (Credit Risk) / Non Financial corporations (NPE- Forbearance)'
tra_cre_irb = tra_cre_irb.replace({'exposure': 
                                       {col: 'Corporates', 
                                       'TOTAL': 'All exposure classes'}})

# Drop country-specific information and non-loss related fields
idx1 = tra_cre_irb['item'].isin(['IRB - Stock of Provisions by IFRS9 Stages', 
                                       'IRB - Exposures by IFRS9 Stages'])
idx2 = tra_cre_irb['country'].isin(['Total / No breakdown'])
losses = tra_cre_irb[idx1 & idx2]

# Pivot table
pivot_df_irb = losses.pivot_table(index=['bank_name', 'item', 'exposure'], 
                                      columns=['period_str', 'scenario'], 
                                      values='amount', 
                                      aggfunc='sum').reset_index()

pivot_df_irb.sample(5)

# Split the dataframe into two based on the item value and clean column lables
stock_df = pivot_df_irb[pivot_df_irb['item'] == 'IRB - Stock of Provisions by IFRS9 Stages'].copy()
exposures_df = pivot_df_irb[pivot_df_irb['item'] == 'IRB - Exposures by IFRS9 Stages'].copy()

stock_df.columns = [f"stock_{i[0]}_{i[1]}" 
                    if index >= 4 and i[1] else (f"stock_{i[0]}" 
                    if index >= 3 else i[0]) for index, i in enumerate(stock_df.columns)]

exposures_df.columns = [f"exposure_{i[0]}_{i[1]}" 
                    if index >= 4 and i[1] else (f"exposure_{i[0]}" 
                    if index >= 3 else i[0]) for index, i in enumerate(exposures_df.columns)]

# Drop the item column as it will not be used in the join
stock_df.drop('item', axis=1, inplace=True)
exposures_df.drop('item', axis=1, inplace=True)

# Merge the two dataframes on bank_name and exposure columns
merged_df = stock_df.merge(exposures_df, on=['bank_name', 'exposure'], how='left')

,bank_name,exposure,stock_2022_12,stock_2023_12_Adverse scenario,stock_2023_12_Baseline scenario,stock_2024_12_Adverse scenario,stock_2024_12_Baseline scenario,stock_2025_12_Adverse scenario,stock_2025_12_Baseline scenario,exposure_2022_12,exposure_2023_12_Adverse scenario,exposure_2023_12_Baseline scenario,exposure_2024_12_Adverse scenario,exposure_2024_12_Baseline scenario,exposure_2025_12_Adverse scenario,exposure_2025_12_Baseline scenario
276,Bayerische Landesbank,Corporates - SME,191.47000,643.318898,214.874823,749.596180,241.140789,803.107961,266.102452,27351.110000,27351.110000,27351.110000,27351.110000,27351.110000,27351.110000,27351.110000
218,Bank Polska Kasa Opieki S.A.,Retail - Secured by real estate property,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
558,Erste Group Bank AG,Retail - Secured by real estate property,503.90000,977.666049,542.082710,1225.109453,598.062005,1611.856079,648.648867,70737.170000,70737.170000,70737.170000,70737.170000,70737.170000,70737.170000,70737.170000
1131,UniCredit S.p.A.,Retail,2668.08966,3943.031959,2592.888914,4497.299430,2780.147240,4817.995554,2956.724689,125972.337696,125972.337696,125972.337696,125972.337696,125972.337696,125972.337696,125972.337696
725,Jyske Bank A/S,Retail - Other - Non SME,39.73930,91.991893,57.438966,106.826294,63.515173,119.369245,69.484429,2117.681500,2117.681500,2117.681500,2117.681500,2117.681500,2117.681500,2117.681500


In [9]:
# Initialize an empty list to store the new rows
new_rows = []

# Calculate ratios for Other template
for _, row in merged_df.iterrows():
    # Exclude observations where calculating ratio is not possible
    if (row['stock_2022_12'] in {0, None}) | (row['stock_2023_12_Adverse scenario'] in {0, None}):
        continue
    
    elif row['exposure'] == 'Corporates - SME':
        ratio = row['stock_2023_12_Baseline scenario'] / row['stock_2023_12_Adverse scenario']
        new_rows.append([row['bank_name'], 'Corporate_SME_impairment_stock_baseline_vs_adverse_Y1', ratio])
        
    elif row['exposure'] == 'Retail':
        ratio = row['stock_2023_12_Baseline scenario'] / row['stock_2023_12_Adverse scenario']
        new_rows.append([row['bank_name'], 'Retail_IRB_impairment_stock_baseline_vs_adverse_Y1', ratio])

    else:
        new_rows.append(['Problem', 'Problem', 0])

# Convert the list to a new dataframe
losses_df = pd.DataFrame(new_rows, columns=['bank_name', 'Metric', 'value'])

,bank_name,Metric,value
0,Problem,Problem,0.000000
1,Problem,Problem,0.000000
2,ABN AMRO Bank N.V.,Corporate_SME_impairment_stock_baseline_vs_adv...,0.551658
3,Problem,Problem,0.000000
4,Problem,Problem,0.000000
...,...,...,...
564,Problem,Problem,0.000000
565,Problem,Problem,0.000000
566,de Volksbank N.V.,Retail_IRB_impairment_stock_baseline_vs_advers...,0.305784
567,Problem,Problem,0.000000


### E. Create a summary

In [10]:
# Vertically concatenate derined datasets
final_df = pd.concat([general_df, losses_df], axis=0).reset_index(drop=True)

In [20]:
# Filter for the any bank in the list and remove empty rows
bank_name = 'Bayerische Landesbank'
final_df = final_df[final_df['Metric'] != "Problem"]

# Calculate the desired quantiles for each group
quantiles = [0, 0.10, 0.25, 0.50, 0.75, 0.90, 1]
quantiles = final_df.groupby('Metric')['value'].quantile(quantiles).reset_index()

# Pivot the table to get the desired format
result = quantiles.pivot(index='Metric', columns='level_1', values='value').reset_index()
result.columns = ['Metric', 'Min', '10%', '25%', '50%', '75%', '90%', 'Max']

bank_df = final_df[final_df['bank_name'] == bank_name]
result_df = result.merge(bank_df[['Metric', 'value']], on='Metric', how='left')
result_df.rename(columns={'value': 'Bank ABC'}, inplace=True)
desired_order = ['Metric', 'Bank ABC', 'Min', '10%', '25%', '50%', '75%', '90%', 'Max']
result_df = result_df[desired_order]

def color_cells(row):
    """
    Color values in 'bank_name_' based on comparison with deciles.
    """
    value = row['Bank ABC']

    # Determine the color for the 'bank_name_' column
    if value < row['10%']:
        return 'color: red'
    elif row['10%'] <= value < row['25%']:
        return 'color: yellow'
    elif row['25%'] <= value < row['50%']:
        return 'color: green'
    elif row['50%'] <= value < row['75%']:
        return 'color: green'
    elif row['75%'] <= value < row['90%']:
        return 'color: green'
    elif row['90%'] <= value < row['Max']:
        return 'color: green'
    else:
        return 'color: darkgreen'
    
    return colors

# Use apply to get the colors for column 'A' based on values in column 'B'
colors = result_df.apply(color_cells, axis=1)

# Apply the color styles to the DataFrame
styled = result_df.style.apply(lambda x: colors, subset=['Bank ABC']).format({
    'Bank ABC': "{:.2%}",
    '10%': "{:.2%}",
    '25%': "{:.2%}",
    '50%': "{:.2%}",
    '75%': "{:.2%}",
    '90%': "{:.2%}",
    'Min': "{:.2%}",
    'Max': "{:.2%}",
})

print(f'   Benchmarking Bank ABC against the EBA 2023 Stresstest results')
styled

   Benchmarking Bank ABC against the EBA 2023 Stresstest results


,Metric,Bank ABC,Min,10%,25%,50%,75%,90%,Max
0,CET1_ratio_adverse_Y1,11.05%,3.09%,8.79%,10.27%,12.14%,14.29%,16.11%,23.18%
1,Corporate_SME_impairment_stock_baseline_vs_adverse_Y1,33.40%,1.13%,27.92%,49.59%,66.29%,81.12%,83.27%,87.65%
2,Leverage_ratio_adverse_Y1,3.36%,1.61%,3.49%,3.95%,4.50%,6.03%,6.93%,12.91%
3,Net_fee_and_commission_income_adverse_vs_baseline_Y1,77.32%,43.42%,64.44%,69.96%,73.46%,77.78%,82.19%,100.00%
4,Net_interest_income_adverse_vs_baseline_Y1,73.98%,-1570.27%,56.18%,67.98%,73.72%,81.60%,88.32%,177.83%
5,Retail_IRB_impairment_stock_baseline_vs_adverse_Y1,47.60%,4.39%,31.70%,51.67%,65.11%,77.99%,87.96%,95.27%
6,TREA_baseline_vs_adverse_Y1,91.67%,83.19%,89.84%,94.02%,97.76%,99.70%,100.74%,101.78%
